In [51]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("./flight_data/1987.csv")

In [12]:
df_c = pd.DataFrame()
for i in range(1987, 2009):
    try:
        df_temp = pd.read_csv("./flight_data/{}.csv".format(i))
    except:
        df_temp= pd.read_csv("./flight_data/{}.csv".format(i), encoding = "ISO-8859-1", low_memory=False)
    df_delay = df_temp[df_temp["ArrDelay"] > 0]
    df_max = df_delay.groupby(by="UniqueCarrier")["ArrDelay"].agg(max)
    df_max.name = i
    df_c = pd.concat([df_c, df_max],axis=1)

ex2_b = df_c.max(axis= 1)
print(ex2_b)

2001
2002


/var/folders/nw/prlspygx479dwlhvnc1kjj8r0000gp/T/ipykernel_38864/2362377259.py:4: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv("./flight_data/{}.csv".format(i))


In [19]:
tmp = df_c.max(axis= 1)
tmp

AA        1525.0
AS        1139.0
CO        1178.0
DL        1189.0
EA        1179.0
HP        1323.0
NW        2598.0
PA (1)    1438.0
PI        1381.0
PS        1033.0
TW         931.0
UA        1612.0
US        1073.0
WN         889.0
ML (1)     584.0
AQ         623.0
B6        1392.0
DH        1438.0
EV        1187.0
FL        1175.0
HA        1309.0
MQ        1707.0
OO        1435.0
TZ        1300.0
XE         939.0
OH        1380.0
F9         920.0
YV         715.0
9E        1942.0
dtype: float64

# PATTERN


In [41]:
df = pd.read_csv("./flight_data/2007.csv")

In [42]:
df = df[["DayOfWeek", "DepTime", "CRSDepTime", "ArrTime", "CRSArrTime", "UniqueCarrier", "DepDelay"]]
df = df.dropna(how='any',axis =0)

In [43]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(df[["DayOfWeek"]])
encoded_data = encoder.transform(df[["DayOfWeek"]])

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())

transformed_df = pd.concat([df[["DepTime", "CRSDepTime", "ArrTime", "CRSArrTime", "UniqueCarrier", "DepDelay"]], encoded_df], axis=1)

transformed_df["Ln_DepDelay"] = np.log(df["DepDelay"])


/Users/shawn/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/shawn/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### OHE UniqueCarrier

In [47]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(df[['UniqueCarrier']])
encoded_data = encoder.transform(df[['UniqueCarrier']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())
transformed_df = pd.concat([transformed_df, encoded_df], axis=1)

In [48]:
transformed_df.columns

Index(['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier',
       'DepDelay', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
       'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'Ln_DepDelay',
       'UniqueCarrier_9E', 'UniqueCarrier_AA', 'UniqueCarrier_AQ',
       'UniqueCarrier_AS', 'UniqueCarrier_B6', 'UniqueCarrier_CO',
       'UniqueCarrier_DL', 'UniqueCarrier_EV', 'UniqueCarrier_F9',
       'UniqueCarrier_FL', 'UniqueCarrier_HA', 'UniqueCarrier_MQ',
       'UniqueCarrier_NW', 'UniqueCarrier_OH', 'UniqueCarrier_OO',
       'UniqueCarrier_UA', 'UniqueCarrier_US', 'UniqueCarrier_WN',
       'UniqueCarrier_XE', 'UniqueCarrier_YV'],
      dtype='object')

In [49]:
features = ['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime',
            'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
            'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
            'UniqueCarrier_9E', 'UniqueCarrier_AA', 'UniqueCarrier_AQ',
            'UniqueCarrier_AS', 'UniqueCarrier_B6', 'UniqueCarrier_CO',
            'UniqueCarrier_DL', 'UniqueCarrier_EV', 'UniqueCarrier_F9',
            'UniqueCarrier_FL', 'UniqueCarrier_HA', 'UniqueCarrier_MQ',
            'UniqueCarrier_NW', 'UniqueCarrier_OH', 'UniqueCarrier_OO',
            'UniqueCarrier_UA', 'UniqueCarrier_US', 'UniqueCarrier_WN',
            'UniqueCarrier_XE', 'UniqueCarrier_YV']

data_features = transformed_df[features]
scaler = StandardScaler()
normalized_features = scaler.fit_transform(data_features)
normalized_data = pd.DataFrame(normalized_features, columns=features)
normalized_data = pd.concat([normalized_data, transformed_df.drop(features, axis=1)], axis=1)

### LinearRegression

In [73]:
normalized_data = normalized_data.dropna(how='any')
# normalized_data = normalized_data[normalized_data["Ln_DepDelay"] > 1][normalized_data["Ln_DepDelay"] < 4]
test_df = normalized_data
X = test_df[features]
y = test_df[["Ln_DepDelay"]]

In [74]:
model = sm.OLS(y.to_numpy(), X.to_numpy())
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):              0.001
Method:                 Least Squares   F-statistic:                              100.5
Date:                Wed, 24 May 2023   Prob (F-statistic):                        0.00
Time:                        19:25:21   Log-Likelihood:                     -4.9172e+06
No. Observations:             2041838   AIC:                                  9.835e+06
Df Residuals:                 2041809   BIC:                                  9.835e+06
Df Model:                          29                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

,const,DepTime,CRSDepTime,ArrTime,CRSArrTime,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,...,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,1.0,-0.223390,-0.224528,-0.278190,-0.319804,2.384976,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,2.299628,-0.248241,-0.201026
1,1.0,1.206122,1.238487,1.105813,1.123197,2.384976,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,2.299628,-0.248241,-0.201026
2,1.0,1.806267,1.722573,1.679524,1.673405,2.384976,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,2.299628,-0.248241,-0.201026
3,1.0,-0.227558,-0.278315,-0.248618,-0.340566,2.384976,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,2.299628,-0.248241,-0.201026
4,1.0,-1.059008,-1.074367,-1.035252,-1.025732,2.384976,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,2.299628,-0.248241,-0.201026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7103415,1.0,0.791438,0.621009,1.482373,1.351585,-0.419291,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,-0.434853,-0.248241,-0.201026
7103416,1.0,0.585138,0.621009,1.320708,1.351585,-0.419291,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,-0.434853,-0.248241,-0.201026
7103418,1.0,0.570552,0.621009,1.283250,1.351585,-0.419291,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,-0.434853,-0.248241,-0.201026
7103421,1.0,0.566384,0.621009,1.330566,1.351585,-0.419291,-0.410772,-0.41294,-0.415207,-0.416322,...,-0.088022,-0.276355,-0.243072,-0.178427,-0.295117,-0.264913,-0.264465,-0.434853,-0.248241,-0.201026


In [65]:
y.describe()

,Ln_DepDelay
count,1.000000e+06
mean,-inf
std,NaN
min,-inf
25%,6.931472e-01
50%,2.302585e+00
75%,3.401197e+00
max,7.424762e+00


In [68]:
np.log(100)

4.605170185988092